<a href="https://colab.research.google.com/github/b4wolf/NEURALNETS/blob/development/graph_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Use torch geometric to make a static graph and then use the LSTMs to feed it on the last iteration and then add continuous temporal measurement

# Important links to use
https://medium.com/@kaoutaramounir/graph-neural-networks-gnns-for-spatial-temporal-climate-modeling-bb2be94d60a0

https://github.com/openclimatefix/graph_weather

https://towardsdatascience.com/graph-convolutional-networks-introduction-to-gnns-24b3f60d6c95


In [ ]:
!pip install ogb
!pip install torch_geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.8/78.8 kB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for littleutils: filename=littleutils-0.2.2-py3-none-any.whl size=7026 sha256=d78bf662e4b97cfa1239a8faca9700e66382a35c64aa50503bc7c776a12373f3
  Stored in directory: /root/.cache/pip/wheels/3d/fe/b0/27a9892da57472e538c7452a721a9cf463cc03cf7379889266
Successfully built littleutils
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 10.9 MB/s eta 0:00:00


In [ ]:
import torch
import torch.nn as nn
import torch_geometric.nn as geom_nn
import pandas as pd
import random
import networkx as nx
from geopy.distance import geodesic
import numpy as np
import torch
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv
import torch.nn.functional as F
import torch.optim as optim
from sklearn.neighbors import NearestNeighbors
from sklearn import preprocessing

ModuleNotFoundError: ignored

In [ ]:
import os, json

path_to_json_files = './WI_data'

# Get all JSON file names as a list
json_file_names = [filename for filename in os.listdir(path_to_json_files) if filename.endswith('.json')]
lat_lon_data = []
targets = []
for json_file_name in json_file_names:
    with open(os.path.join(path_to_json_files, json_file_name)) as json_file:
        json_text = json.load(json_file)
        lat_lon_data.append([json_text['lat_lon'][1], json_text['lat_lon'][0]])
        targets.append(json_text['elevation'])
lat_lon_data = torch.tensor(lat_lon_data, dtype=torch.float)
targets = torch.tensor(targets, dtype=torch.float).reshape(-1,1)

In [ ]:
scaler_lat_lon = preprocessing.StandardScaler().fit(lat_lon_data)
scaler_targets = preprocessing.StandardScaler().fit(targets)

scaled_lat_lon = scaler_lat_lon.transform(lat_lon_data)
scaled_targets = scaler_targets.transform(targets)

In [ ]:
nbrs = NearestNeighbors(n_neighbors=4, algorithm='ball_tree', metric='haversine').fit(scaled_lat_lon)
distances, indices = nbrs.kneighbors(scaled_lat_lon)

In [ ]:
print(indices)

[[ 0 35 27  2]
 [ 1 13  7 18]
 [ 2 40 20  0]
 [ 3 25 18 29]
 [ 4  5 26 36]
 [ 5  4 10 26]
 [ 6 17 42 32]
 [ 7 13 23 39]
 [ 8 28 45 44]
 [ 9 25 15  3]
 [10  5 44 41]
 [11 38 32 19]
 [12 22 32 11]
 [13  7  1 23]
 [14 22 45 12]
 [15 29  9 34]
 [16 21 35 31]
 [17  6 42 46]
 [18  3 25  1]
 [19 38 33 11]
 [20 26 36  4]
 [21 16 31 35]
 [22 12 14 33]
 [23 39  7 13]
 [24 43  9 34]
 [25  3 18  9]
 [26  4 20  5]
 [27 29 35 31]
 [28 37 45  8]
 [29  3 15 27]
 [30 40  2 16]
 [31 39 16 35]
 [32 11 38 19]
 [33 19 38 11]
 [34 37 15  9]
 [35 16 31  0]
 [36 20  4 41]
 [37 34 28 15]
 [38 11 32 19]
 [39 23  7 13]
 [40  2 30 16]
 [41 10  5 36]
 [42  6 17 19]
 [43 24 45 28]
 [44 10  5 33]
 [45 28  8 43]
 [46 42 17 41]]


In [ ]:
import torch
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv
import torch.nn.functional as F
import torch.optim as optim
import numpy as np

# Convert NetworkX graph to PyTorch Geometric data
def convert_to_torch_geometric_data(lat_lon_data, distances, indices):
    # Node features
    x = scaled_lat_lon

    # Edge index
    edge_index = []
    edge_attr = []
    for pts in indices:
      node, neighbors = pts[0], pts[1:]
      for n in neighbors:
        edge_index.append([node, n])
        edge_attr.append([distances[node]])

    edge_index = torch.tensor(edge_index, dtype=torch.float).t().contiguous()
    edge_attr = torch.tensor(edge_attr, dtype=torch.float)

    return Data(x=x, edge_index=edge_index, edge_attr=edge_attr)

data = convert_to_torch_geometric_data(lat_lon_data, distances, indices)

# Create dummy target values (e.g., temperature)
Y = scaled_targets
data.y = Y
print(data.validate(raise_on_error=True))
# Define a simple GCN model
class GCN(torch.nn.Module):
    def __init__(self):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(2, 16)  # 2 input features (lat, lon), 16 output features
        self.conv2 = GCNConv(16, 1)  # 16 input features, 1 output feature (for prediction)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)
        return x.squeeze()

model = GCN()
optimizer = optim.Adam(model.parameters(), lr=0.01)

# Training the model
for epoch in range(1000):
    model.train()
    optimizer.zero_grad()
    out = model(data)
    loss = F.mse_loss(out, data.y)
    loss.backward()
    optimizer.step()

    # Print loss every 50 epochs
    if epoch % 50 == 0:
        print(f'Epoch {epoch}, Loss: {np.sqrt(loss.item())}')

# Prediction
model.eval()
pred = model(data)
print("Ground truth:", Y.detach().numpy())
print("Predictions:", pred.detach().numpy())


True


<ipython-input-7-3b912c5b15d8>:23: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:261.)
  edge_attr = torch.tensor(edge_attr, dtype=torch.float)


TypeError: ignored

In [ ]:
print(data.x)

tensor([[ 43.9844, -88.5570],
        [ 46.5487, -90.9189],
        [ 43.2119, -90.1798],
        [ 44.5451, -89.5303],
        [ 46.6897, -92.0947],
        [ 43.1104, -88.0344],
        [ 44.3592, -89.8369],
        [ 44.6139, -88.7314],
        [ 45.6312, -89.4675],
        [ 42.6149, -89.5904],
        [ 42.9472, -87.8967],
        [ 45.1008, -90.3033],
        [ 43.1696, -88.7232],
        [ 45.1483, -92.5381],
        [ 43.1406, -89.3453],
        [ 43.4265, -88.7038],
        [ 44.9262, -89.6266],
        [ 42.7611, -87.8136],
        [ 44.7776, -89.6668],
        [ 42.6907, -88.3046],
        [ 44.1288, -87.6806],
        [ 43.4222, -88.1279],
        [ 44.8437, -87.4216],
        [ 44.8658, -91.4843],
        [ 46.0253, -91.4442],
        [ 44.6369, -90.1893],
        [ 43.5220, -89.7710],
        [ 43.8800, -91.2500],
        [ 44.4794, -88.1367],
        [ 43.7698, -87.8517],
        [ 43.0193, -91.1237],
        [ 45.1542, -89.1107],
        [ 42.8868, -90.2362],
        [ 

In [ ]:
import torch
import torch.nn as nn
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv
from torch_geometric.loader import DataLoader
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
import random

# Dummy data for illustration
latitude = [val[0] for val in lat_lon_data]
longitude = [val[1] for val in lat_lon_data]
elevation = targets

# Create a simple graph
edge_index = torch.tensor([[random.randint(0, len(elevation)) for i in range(len(elevation))], [random.randint(0, len(elevation)) for i in range(len(elevation))]], dtype=torch.long)
x = torch.tensor(list(zip(latitude, longitude)), dtype=torch.float)
y = torch.tensor(elevation, dtype=torch.float)
print(edge_index)
data = Data(x=x, edge_index=edge_index, y=y)

# Define a simple GNN model
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = GCNConv(2, 16)
        self.conv2 = GCNConv(16, 1)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = torch.relu(x)
        x = self.conv2(x, edge_index)
        return x

# Instantiate the model
model = Net()

# Define loss and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

# Training
for epoch in range(100):
    optimizer.zero_grad()
    out = model(data)
    loss = criterion(out, data.y.view(-1, 1))
    loss.backward()
    optimizer.step()
    print(f'Epoch {epoch + 1}, Loss: {loss.item()}')

# Make predictions
with torch.no_grad():
    model.eval()
    predicted_elevation = model(data).squeeze().numpy()

# Plot the results
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(latitude, longitude, elevation, label='True Elevation')
ax.scatter(latitude, longitude, predicted_elevation, label='Predicted Elevation')
ax.set_xlabel('Latitude')
ax.set_ylabel('Longitude')
ax.set_zlabel('Elevation')
ax.legend()
plt.show()


tensor([[12, 46, 22, 30, 15, 45, 11, 30,  1, 11, 14,  9, 32, 42, 29, 40, 38, 10,
         34, 19, 19,  7, 30, 22, 10, 40, 19, 11, 39,  8, 32, 14, 18, 40, 15, 33,
         15, 26, 33, 15, 22,  1, 26,  5, 31, 13,  9],
        [15,  6, 28, 26, 23, 20,  3,  1, 13,  7,  9, 18, 28,  4, 21, 34, 12, 39,
         28, 19, 34, 37,  8, 12, 42, 10, 33, 14, 40,  0, 33, 23, 17,  7,  1, 32,
         46, 31, 47, 32, 33, 30, 42,  0, 14,  1, 22]])


RuntimeError: ignored

In [ ]:
import os
import json
import pandas as pd
from sklearn.neighbors import NearestNeighbors

def load_weather_coords(path_to_json):
    weather_coords = {}
    for json_file in os.listdir(path_to_json):
        if json_file.endswith('.json'):
            with open(os.path.join(path_to_json, json_file)) as f:
                data = json.load(f)
                weather_coords[json_file.split('.')[0]] = data['lat_lon'] + [data['elevation']]
    return weather_coords

weather_coords = load_weather_coords('WI_weather_coords')
weather_df = pd.DataFrame.from_dict(weather_coords, orient='index', columns=['lat', 'lon', 'elevation'])

def load_weather_data(path_to_csv):
    all_data = []
    for csv_file in os.listdir(path_to_csv):
        if csv_file.endswith('.csv'):
            data = pd.read_csv(os.path.join(path_to_csv, csv_file), parse_dates=[0])
            data['station'] = csv_file.split('.')[0]
            all_data.append(data)
    return pd.concat(all_data)

clean_weather_data = load_weather_data('clean_WI_data')

def load_pm_data(path_to_csv):
    all_data = []
    for csv_file in os.listdir(path_to_csv):
        if csv_file.endswith('.csv'):
            data = pd.read_csv(os.path.join(path_to_csv, csv_file))
            coords = csv_file.replace('processed_monitor_', '').split('_')
            data['lat'] = float(coords[0])
            data['lon'] = float(coords[1])
            all_data.append(data)
    return pd.concat(all_data)

pm_data = load_pm_data('processed_monitor')

nbrs = NearestNeighbors(n_neighbors=1).fit(weather_df[['lat', 'lon']])
distances, indices = nbrs.kneighbors(pm_data[['lat', 'lon']])
pm_data['nearest_station'] = weather_df.iloc[indices.flatten()].index

pm_data['timestamp'] = pd.to_datetime(pm_data['timestamp'])
clean_weather_data['timestamp'] = pd.to_datetime(clean_weather_data['timestamp'])
joined_data = pm_data.merge(clean_weather_data, left_on=['nearest_station', 'timestamp'], right_on=['station', 'timestamp'])

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(14, 7))
plt.scatter(joined_data['timestamp'], joined_data['Value'], alpha=0.6, c='blue')
plt.title('PM Values Over Time')
plt.xlabel('Time')
plt.ylabel('PM Value')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()